<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#All-patients" data-toc-modified-id="All-patients-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>All patients</a></span></li><li><span><a href="#Conditioned-on-gender-(sex)" data-toc-modified-id="Conditioned-on-gender-(sex)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Conditioned on gender (sex)</a></span><ul class="toc-item"><li><span><a href="#Male" data-toc-modified-id="Male-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Male</a></span></li><li><span><a href="#Female" data-toc-modified-id="Female-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Female</a></span></li></ul></li><li><span><a href="#Conditioned-on-age" data-toc-modified-id="Conditioned-on-age-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conditioned on age</a></span><ul class="toc-item"><li><span><a href="#Young-(1-19-years)" data-toc-modified-id="Young-(1-19-years)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Young (1-19 years)</a></span></li><li><span><a href="#Adult-(20-65-years)" data-toc-modified-id="Adult-(20-65-years)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Adult (20-65 years)</a></span></li><li><span><a href="#Elderly-(<65-years)" data-toc-modified-id="Elderly-(<65-years)-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Elderly (&lt;65 years)</a></span></li></ul></li><li><span><a href="#Save-all-the-populations-in-disproportionality-estimation" data-toc-modified-id="Save-all-the-populations-in-disproportionality-estimation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Save all the populations in disproportionality estimation</a></span></li></ul></div>

# Load data


In [1]:
import itertools
from tqdm import tqdm
from collections import Counter
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# %matplotlib notebook
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

# load the dictionaries for drugs, SE, indications
drug_dic = pickle.load(open('../Data/high_order/dic/drug_dic_FAERS.pk', 'rb'))  # Save it as dataframe, pandas format
SE_dic = pickle.load(open('../Data/high_order/dic/side_dic_FAERS.pk', 'rb'))  # Save it as dataframe, pandas format

# In this MeDRA_dic, key is string of PT_name, value is a list:
# [PT, PT_name, HLT,HLT_name,HLGT,HLGT_name,SOC,SOC_name,SOC_abbr]
meddra_se_disease_dic = pickle.load(open('../Data/high_order/dic/MeDRA_dic.pk', 'rb'))
MedDRA_dic_all = pickle.load(open('../Data/high_order/dic/MedDRA_dic_all.pk', 'rb'))

In [2]:
def format_tex(float_number):
    exponent = np.floor(np.log10(float_number))
    mantissa = float_number/10**exponent
    mantissa_format = str(mantissa)[0:3]
    if float_number!=0:
        return "$< {0}\times10^{{{1}}}$".format(mantissa_format, str(int(exponent)))
    else:
        return "$< 0 \times10^{0}$"
    
def weird_division(n, d):
    return n / d if d else 0

def CI(ROR, A, B, C, D):
    ror = np.log(ROR)
    sq = 1.96*np.sqrt(weird_division(1, A) + weird_division(1, B) + weird_division(1, C) +weird_division(1, D))
    CI_up = np.exp(ror + sq)
    CI_down = np.exp(ror - sq)
    return CI_up, CI_down

# All patients

In [3]:
SE_uncondition = pickle.load(open('../Data/pandemic/SE_uncondition.pk', 'rb'))

In [4]:
 # Remove some SE that make less sense or not related to medication
# """15765: device use error, 2688:device physical property issue; 2232:product contamination;4293: compulsions; 
# 10484: infusion; 6275:body height below normal; 2325:large for dates baby. 10870:product distribution issue.
# 8657:poverty of thought content. 2222:poor personal hygiene. 1039:family stress. 1215:nosocomial infection.
# 4347:syringe issue. 4374:confabulation, 647:device occlusion, 6141:product outer packaging issue
# 3716:product contamination with body fluid, 2048:fear of disease. 10249:drain placement.1848:treatment failure
# 2659:device leakage. 8613:device alarm issue. 9141:product label confusion. 8593:device connection issue
# 3820:application site discharge. 6799:post procedural discharge: 1728:poisoning deliberate
# 2576:social problem. 132:device malfunction  2591:needle issue. 4216: exomphalos. 1669:fear of falling
# 713:medical device change. 4425:intercepted medication error. 1809:exposure via partner. 3087 :liquid product physical issue
# 4005:medical device implantation. 5616:application site discomfort. 865:device failure. 4908:device ineffective
# 14262:reproductive complication associated with device. 4624: device colour issue。 1051 educational problem
# 1281:device difficult to use. 4551:pregnancy of partner. 9615:prescribed underdose. 11243:product physical consistency issue
# 1598:product odour abnormal. 437:accident at work. 1451:product packaging quantity issue,1679:incorrect drug administration rate
# 627:hospice care. 238:unevaluable event. 3067: imprisonment. 8012:stress at work  6177:medical device pain
# 277:mass, 1046:thrombosis in device, 905:product size issue. 2071:product label issue. 218:off label use
# 2744:product colour issue.  1224: laboratory test abnormal 2139:product packaging issue. 5240:product contamination physical
# 11139: expired device used. 12123:lack of injection site rotation. 639:device issue. 1095:injury associated with device
# 1747:therapeutic product ineffective, 7592: product dropper issue. 158:incorrect dose administered. 1041:economic problem
# 341:device related infection. 655: product physical issue. 4257:device related sepsis,968:treatment noncompliance
# 353:road traffic accident. 991:medication error. 335:drug ineffective, 14157:device physical property issue, 
# 14881:device power source issue, 11494:off label use, 13223: device malfunction, 15755:unintentional medical device removal
# 13242:drug dependence, 
# r *hallucination, visual* with code :4652, malaise:4515, condition aggravated:4846
# 15201:toxicity to various agents,
# 'eating disorder', 'incoherent', 'out of specification test results', 'antibody test negative', 'gene mutation identification test positive', 'gun shot wound', 
# 'bed sharing','antibody test positive', 'large for dates baby,viral load', 'small for dates baby', 'x-ray', 
# 'scan', 'blood test','female condom', 'sleep study','boredom',' toxicity to various agents',
# 'transplant failure', 'pregnancy after post coital contraception','drug intolerance', 'drug withdrawal syndrome'
# [732,7823,15255,12614,13146,1347,12821,7349,1117,4512,3818,315]
# 521:infection

# """

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
drop_list_word = [15702,15672,11262,13634,11488,14158,12935,15656,15342,15351,5024,14156,13487,14707,13509,12268,
             4771,2914,14679,7943,14884,9236,12683,14864,14891,14874,14701,13901,14705,9229,14697,14704,14680,
             14673,15346,12579,15350,4306,2977,13504,14689,14743,11733,14654,14674,8616,14676,14669,15341,14665,
             9503,1664,14668,2343,14155,15147,14700,15118,15004,14671,15768,11378,14709,15148,15345,14889,5329,
             14670,10528,13476,15306,3863,15344,14675,14656,14695,14863,15623,5344,15047,14666,14691,15707,14687,
             14887,8966,13674,12425,7750,15622,14657,7706,9555,14681,14686,14885,14496,544,7074,15703,15598,
             14690,15126,12188,14880,7384,14703,8432,15274,14682,15085,10945,15164,14678,15343,15645,14692,
             14696,15087,12874,14698,14677,8261,2095,802,13601,14871,14683,15705,14688,15163,12242,14694,14484,
             11838,15524,14693,14318,6939,14655,14882,14706,7749,15730,14888,12580,868,15347,15265,13693,14667,
             15799,15842,15912,15704,15751,5005,14877,15046,15379,15706,15048,12546,15112,12547,
             7145,2417,3154,3173,3084,8691,3124,9478,7657,12659,13943,7131,3142,8823,7660,10212,11438,9118,9107,
                  3043,13974,6836,12286,7832,960,11589,11452,11368,11373,10669,13548,10204,7647,12419,3141,10406,
                  7651,10658,7661,10372,12173,6228,13716,10607,12387,10405,10403,876,13213,7360,10890,4544,10889,
                  4494,7654,13229,7736,11478,10550,13513,13212,11776,7817,7653,11339,11369,14238,12241,13717,11337,
                  7659,14138,3073,12174,7652,13215,11335,11390,9486,3135,10404,7655,14039,12201,12184,13812,2883,
                  3163,12086,14091,11742,13314,8919,10822,3028,4713,7658,12924,11130,13205,13782,14531,14139,14141,
                  10424,11682,7835,7662,15470,13203,13202,13316,12224,13713,13774,13934,11457,4714,13209,13321,13325,
                  11661,13305,10423,13187,13303,4633,14371,3045,14370,7934,13216,12202,13775,13311,11631,11632,11451,
                  12508,10821,13208,14515,11660,13301,10661,13779,12923,11131,8168,13806,10315,13206,13201,11610,13322,
                  961,13714,13308,13312,13326,13204,13814,14814,13330,10667,11341,10402,13837,12258,13368,13210,13896,
                  13327,10425,11453,14601,13709,10593,11338,10456,13742,13815,13228,13841,13319,13673,7656,11540,13784,
                  7650,7649,11342,13894,13324,13688,13923,10666,13711,15686,11906,15689,11559,12479,12509,13310,13214,
                  576,15681,13317,8670,3871,11336,15729,15841,13743,11343,8958,15846,15925,10356,13977,15854,15861,15870,
                  15874,11725,15839,15864,13778,15857,14802,15856,15924,15867,15690,13710,13329,15876,15859,15872,15873,
                  15868,15863,15816,15804,13318,13741,15855,15922,14588,13808,15809,13342,697,13422,8784,2072,3258,10815,
                  10230,11855,12160,646,7582,1875,2616,8294,3079,9208,10217,3169,14389,2078,8463,10292,10825,8250,2615,
                  12252,14451,10387,699,11101,13413,5727,3138,3078,4500,3182,13189,701,13949,4799,10323,14110,13424,10567,
                  10788,1751,14390,2079,14251,10651,10083,10640,10296,700,10475,4696,12002,11105,7717,8897,10681,2073,11606,
                  12505,12506,12934,6859,9309,5100,5654,12483,7614,15552,10297,5216,10436,11104,13425,14090,5860,4303,12484,
                  7221,10904,4304,10298,7716,5205,7259,12382,6369,15740,13925,11466,13426,13641,13875,11261,15826,10056,15533,
                 2649,11316,1059,14220,11492,2734,12902,2735,2732,8238,2428,7056,8027,10490,10912,5234,10519,9819,11479,4970,
                  11489,12552,7283,5162,10574,3765,1545,9241,1208,13986,1101,12338,1531,12120,10775,14723,2550,12941,557,6439,
                  1421,2507,6985,7994,12035,310,10782,7036,12284,13169,8370,13976,1840,2736,13819,7471,12009,1884,9531,11902,
                  11560,13832,14406,12339,4023,12553,1103,6438,3764,13700,10445,8050,2010,12243,10396, 12649,
                 3252, 5976, 13183, 12597]
drop_list = [15765, 2688, 2232,  4293,  10484,  6275, 2325,10870, 8657, 2222, 1039,1215,
             4347, 4374, 647 ,6141,3716, 2048 ,10249 ,1848, 2659, 8613, 9141, 8593, 3820 ,6799, 1728 ,2576,
             132, 2591,4216, 1669, 713, 4425, 1809, 3087 ,4005 ,5616, 865 ,4908, 14262, 4624 ,1051, 1281,
             4551, 9615, 11243 ,1598, 437, 1451, 1679, 627, 238, 3067, 8012, 6177, 277, 1046 ,905 ,2071,
             218, 2744, 1224 ,2139 ,5240, 11139, 12123, 639, 1095, 1747, 7592,158, 1041, 341, 655, 4257,
             968 ,353, 991 ,335,14157, 14881,11494, 13223,15755,3809,698, 7807,7715,
            9503,14665,14666,2875,8725,10205,7888,13487,15147,7648,15344,15346,2914,7943,
            14871,214,220,472,515,13242, 15201,732,7823,15255,12614,13146,1347,12821,7349,1117,4512,3818,315, 
             521,4652, 4515, 4846]

drop_list.extend(drop_list_word)

## generate corresponding meddra ID from the SE_code.

# In this MeDRA_dic, key is string of PT_name, value is a list:
# [PT, PT_name, HLT,HLT_name,HLGT,HLGT_name,SOC,SOC_name,SOC_abbr]
meddra_se_disease_dic = pickle.load(open('../Data/high_order/dic/MeDRA_dic.pk', 'rb'))
# key is code, [name, umls ID]
SE_code_dic = pickle.load(open('../Data/high_order/dic/SE_code_dic.pk', 'rb'))
drop_SE_name = []
drop_SE_meddra = []
for i in drop_list:
    na = SE_code_dic[i][0]
    
    try: # if this SE don't has meddra ID, then continue
        m_id = meddra_se_disease_dic[na][0]
    except:
        continue    
    drop_SE_meddra.append(m_id)
    drop_SE_name.append(na)

meddra_drop_list = ['10003051', '10003054', '10022064', '10022094', '10027091', '10048396', '10050114', 
                    '10050464', '10050729', '10053425', '10053998', '10054846', '10054994', '10057581', 
                    '10057843', '10058142', '10058974', '10059058', '10061111', '10061153', '10061649', 
                    '10063781', '10063866', '10064355', '10064366', '10064382', '10064385', '10064505', 
                    '10064998', '10065117', '10065484', '10066967', '10068003', '10068383', '10069216', 
                    '10069842', '10069902', '10071430', '10072720', '10073303', '10073336', '10074425', 
                    '10074495', '10074497', '10074498', '10074508', '10074555', '10074586', '10074704', 
                    '10074758', '10074796', '10074853', '10074860', '10074868', '10074896', '10074902', 
                    '10074903', '10074904', '10074905', '10074906', '10074946', '10075097', '10075103',
                    '10075107', '10075333', '10075373', '10075461', '10075511', '10075571', '10075573', 
                    '10075574', '10075578', '10075580', '10075585', '10075765', '10075928', '10075933', 
                    '10075965', '10075967', '10075971', '10076053', '10076065', '10076070', '10076073', 
                    '10076087', '10076089', '10076091', '10076101', '10076128', '10076133', '10076141', 
                    '10076182', '10076232', '10076273', '10076308', '10076309', '10076368', '10076470', 
                    '10076476', '10076481', '10076503', '10076542', '10076544', '10076573', '10076637', 
                    '10076639', '10076869', '10076874', '10076897', '10076936', '10076991', '10077040', 
                    '10077107', '10077455', '10077643', '10077659', '10077672', '10077678', '10077767', 
                    '10077796', '10077800', '10077801', '10077812', '10078105', '10078156', '10078325', 
                    '10078340', '10078390', '10078504', '10078525', '10078668', '10078675', '10079007', 
                    '10079078', '10079212', '10079213', '10079221', '10079277', '10079315', '10079316', 
                    '10079317', '10079381', '10079400', '10079404', '10079466', '10079523', '10079645', 
                    '10079843', '10079846', '10079849', '10079903', '10080000', '10080001', '10080092', 
                    '10080099', '10080179', '10080231', '10080304', '10080357', '10080359', '10080459',
                    '10080648', '10080714', '10080718', '10080751', '10080753', '10080754', '10080804', 
                    '10080901', '10080903', '10080974', '10081202', '10081301', '10081359', '10081478', 
                    '10081479', '10081480', '10081540', '10081572', '10081574', '10081575', '10081576', 
                    '10081577', '10081578', '10081579', '10081580', '10081581', '10081675', '10081704', 
                    '10081742', '10081743', '10081770', '10081771', '10082169', '10082200', '10082201', 
                    '10082202', '10082204', '10082205', '10082292', '10082458', '10082527', '10083420', 
                    '10083599', '10083995', '10061427' ,'10002653',  '10077122',  '10071095', '10001756', 
                    '10002730', '10008453', '10014404', '10025250', '10034998', '10037794', '10051082',
                    '10051083', '10052909', '10053073', '10053468', '10053469', '10054976', '10054977', '10056613', 
                    '10057374', '10057480', '10058909', '10059283', '10059828', '10059862', '10061018', '10061758', 
                    '10062035', '10062117', '10064728', '10065100', '10065154', '10065357', '10066377', '10066401', '10067768', 
                    '10068048', '10068492', '10072806', '10074079', '10074300', '10074746', '10074842', '10074950', '10074982', 
                    '10078115', '10078798', '10079637', '10080422', '10083202'
                   ]
# '10084268', '10051905', '10084271','10084451','10070255','10084380','10016256',
##  '10084268' is covid, we remove it to [with all other explicity covid symptoms?]
## '10051905': 'coronavirus infection'
## '10084271'	'sars-cov-2 test positive'
## '10084451'	'suspected covid-19'	
## 10070255	coronavirus test positive	
## 10084380	covid-19 pneumonia	


## 10077122	device delivery system issue
## '10016256'	'fatigue'	
## '10071095'	'growth failure'	

drop_SE_meddra.extend(meddra_drop_list)
pickle.dump(drop_SE_name, open('../Data/pandemic/drop_SE_name.pk','wb'))
drop_list = drop_SE_meddra

In [6]:
SE_uncondition = SE_uncondition.drop_duplicates('SE')

idd = [i not in drop_list for i in SE_uncondition['SE']]
SE_uncondition = SE_uncondition[idd]

"""Nan = 0/0, in our case means nothing, so we drop them first."""
SE_uncondition = SE_uncondition[SE_uncondition['2019_ROR'].notna()]

In [7]:
# """Find the ID of nonsense SE by keywords, and then copy the IDs to the above drop_list"""
# """Remove the SE with specific word"""
# # drop_word = ['device', 'issue', 'product', 'equipment', 'exposure', 'broken','falling', 'suicide','idea', 'site',
# #             'crime', 'foreign', 'quality','drug','pregnancy','dose', 'nonspecific' ,'homicid','event','wound',
# #              'idea', 'transplant', 'thoughts', 'user','infusion', 'plague', 'technique', 'medication']
# drop_word = ['therapy']

# drop_index = [any(word in se for word in drop_word) for se in SE_uncondition.name]
# drop_list_1 = SE_uncondition[drop_index]
# SE_uncondition.shape, drop_list_1.shape

# print(list(drop_list_1.SE))

# ll = ['eating disorder', 'incoherent', 'out of specification test results', 'antibody test negative', 'gene mutation identification test positive', 'gun shot wound', 
# 'bed sharing','antibody test positive', 'large for dates baby,viral load', 'small for dates baby', 'x-ray', 
# 'scan', 'blood test','female condom', 'sleep study','boredom',' toxicity to various agents',
# 'transplant failure', 'pregnancy after post coital contraception','drug intolerance', 'drug withdrawal syndrome', 'gustatory and olfactory', 'anosmia']
# for i in ll:
#     print(list(SE_uncondition[SE_uncondition.name==i].SE))


In [8]:
SE_uncondition.head(3)

,SE,name,2013_A,2013_B,2014_A,2014_B,2015_A,2015_B,2016_A,2016_B,2017_A,2017_B,2018_A,2018_B,2019_A,2019_B,2020_A,2020_B,2013_ROR,2014_ROR,2015_ROR,2016_ROR,2017_ROR,2018_ROR,2019_ROR,2013_Delta,2014_Delta,2015_Delta,2016_Delta,2017_Delta,2018_Delta,2019_Delta
1,10000029,5-alpha-reductase deficiency,0,89334,0,97804,0,179383,0,169233,0,210736,0,244005,0,220920,1,211151,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,10000044,abdomen crushing,0,89334,0,97804,0,179383,0,169233,0,210736,0,244005,0,220920,1,211151,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
3,10000050,abdominal adhesions,14,89320,14,97790,16,179367,7,169226,21,210715,16,243989,15,220905,12,211140,0.362603,0.396988,0.637138,1.373978,0.570278,0.866684,0.836999,-0.142857,-0.142857,-0.25,0.714286,-0.428571,-0.25,-0.2


In [9]:
SE_uncondition_2019 = SE_uncondition[['SE','name','2019_A', '2019_B', '2020_A','2020_B','2019_ROR','2019_Delta']]
SE_uncondition_2019['p_value'] = SE_uncondition_2019.apply(lambda row: stats.fisher_exact([[row['2019_A'], row['2019_B']], [row['2020_A'], row['2020_B']]])[1], axis = 1)

# multipletests
SE_uncondition_2019['sig'], SE_uncondition_2019['p_corrected']  = multipletests(pvals=SE_uncondition_2019['p_value'], alpha=0.05, method='bonferroni')[0:2]
# calculate 95% confidential interval


### for volcano plot, keep the ROR and P-value of all SE
pickle.dump(SE_uncondition_2019, open('../Data/pandemic/SE_uncondition_2019_volcano.pk', 'wb'))  # update the dataframe with ROR and Delta

SE_uncondition_2019_sig = SE_uncondition_2019[SE_uncondition_2019['sig']==True]
SE_uncondition_2019_sig['CI_upper'] = SE_uncondition_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[0], axis = 1)
SE_uncondition_2019_sig['CI_lower'] = SE_uncondition_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[1], axis = 1)

print('the figure for volcano figure is saved')

the figure for volcano figure is saved


In [11]:
SE_uncondition_2019_sig_over = SE_uncondition_2019_sig[SE_uncondition_2019_sig['2019_Delta']>0]
SE_uncondition_2019_sig_under = SE_uncondition_2019_sig[SE_uncondition_2019_sig['2019_Delta']<0]

SE_uncondition_2019_sig_under.sort_values('p_corrected', ascending=True).head()

,SE,name,2019_A,2019_B,2020_A,2020_B,2019_ROR,2019_Delta,p_value,sig,p_corrected,CI_upper,CI_lower
10922,10064848,chronic kidney disease,2113,218807,640,210512,0.314822,-0.697113,2.404883e-169,True,2.395504e-165,0.344000,0.288118
14808,10079622,sprue-like enteropathy,353,220567,0,211152,0.000000,-1.000000,2.504882e-103,True,2.495112e-99,0.000000,0.000000
9730,10061481,renal injury,556,220364,127,211025,0.238526,-0.771583,6.892114e-61,True,6.865235e-57,0.289263,0.196688
14358,10077512,end stage renal disease,585,220335,213,210939,0.380321,-0.635897,1.674309e-37,True,1.667779e-33,0.444959,0.325073
12565,10070608,infective pulmonary exacerbation of cystic fib...,661,220259,329,210823,0.520008,-0.502269,2.607747e-23,True,2.597577e-19,0.593610,0.455532


# Conditioned on gender (sex)
 
The gender contains:
- Male
- Female
- unknown

So that the sum of male and female may not equals to the uncondition. In analysis, we omit unknown gender.

## Male

In [12]:
SE_male = pickle.load(open('../Data/pandemic/SE_male.pk', 'rb'))
idd_male = [i not in drop_list for i in SE_male['SE']]  # drop the nonsense SE
SE_male = SE_male[idd_male]

SE_male = SE_male.drop_duplicates('SE')
SE_male = SE_male[SE_male['2019_ROR'].notna()]
SE_male_2019 = SE_male[['SE','name','2019_A', '2019_B', '2020_A','2020_B','2019_ROR','2019_Delta']]
SE_male_2019['p_value'] = SE_male_2019.apply(lambda row: stats.fisher_exact([[row['2019_A'], row['2019_B']], [row['2020_A'], row['2020_B']]])[1], axis = 1)

# multipletests
SE_male_2019['sig'], SE_male_2019['p_corrected']  = multipletests(pvals=SE_male_2019['p_value'], alpha=0.05, method='bonferroni')[0:2]
SE_male_2019_sig = SE_male_2019[SE_male_2019['sig']==True]
SE_male_2019_sig['CI_upper'] = SE_male_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[0], axis = 1)
SE_male_2019_sig['CI_lower'] = SE_male_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[1], axis = 1)

SE_male_2019_sig.sort_values('p_corrected', ascending=True)

,SE,name,2019_A,2019_B,2020_A,2020_B,2019_ROR,2019_Delta,p_value,sig,p_corrected,CI_upper,CI_lower
12427,10084268,covid-19,0,73675,667,72066,inf,inf,4.706835e-204,True,3.385155e-200,inf,inf
6020,10051905,coronavirus infection,0,73675,243,72490,inf,inf,1.202294e-74,True,8.646896e-71,inf,inf
143,10001551,alanine aminotransferase increased,185,73490,658,72075,3.626584,2.556757,1.324135e-64,True,9.523179e-61,4.270457,3.079790
2205,10019063,hallucination,318,73357,870,71863,2.792726,1.735849,6.791015e-62,True,4.884098e-58,3.176776,2.455105
4125,10038669,respiratory arrest,31,73644,250,72483,8.193690,7.064516,1.336399e-44,True,9.611384e-41,11.902156,5.640705
3300,10029331,neuropathy peripheral,353,73322,803,71930,2.318810,1.274788,1.470464e-42,True,1.057558e-38,2.629103,2.045137
387,10003481,aspartate aminotransferase increased,170,73505,487,72246,2.914628,1.864706,2.090067e-37,True,1.503176e-33,3.471706,2.446940
12429,10084271,sars-cov-2 test positive,0,73675,108,72625,inf,inf,1.474445e-33,True,1.060421e-29,inf,inf
1424,10011906,death,3749,69926,4742,67991,1.300868,0.264871,9.975244e-32,True,7.174195e-28,1.359543,1.244726
2443,10020983,hypogammaglobulinaemia,5,73670,110,72623,22.317172,21.000000,2.219081e-27,True,1.595963e-23,54.688511,9.107145


In [13]:
SE_male_2019_sig_over = SE_male_2019_sig[SE_male_2019_sig['2019_Delta']>0]
SE_male_2019_sig_under = SE_male_2019_sig[SE_male_2019_sig['2019_Delta']<0]

SE_male_2019_sig_under.sort_values('p_corrected', ascending=True).head()

,SE,name,2019_A,2019_B,2020_A,2020_B,2019_ROR,2019_Delta,p_value,sig,p_corrected,CI_upper,CI_lower
9069,10064848,chronic kidney disease,336,73339,158,72575,0.475188,-0.529762,2.275878e-15,True,1.636811e-11,0.574247,0.393218
2135,10018687,granulocytopenia,87,73588,18,72715,0.209381,-0.793103,5.350892e-12,True,3.848361e-08,0.347853,0.126031
1881,10016256,fatigue,2884,70791,2371,70362,0.827135,-0.177878,1.653434e-11,True,1.189150e-07,0.874188,0.782614
1397,10011762,cystic fibrosis,76,73599,19,72714,0.253043,-0.750000,3.166513e-09,True,2.277356e-05,0.418381,0.153044
10352,10070608,infective pulmonary exacerbation of cystic fib...,249,73426,133,72600,0.540214,-0.465863,5.534627e-09,True,3.980504e-05,0.666953,0.437558


In [14]:
l_male = list(SE_male_2019_sig.SE)
l_uncondition = list(SE_uncondition_2019_sig.SE)
set(l_male) - set(l_uncondition)

{'10016256',
 '10017788',
 '10018800',
 '10019211',
 '10024378',
 '10028810',
 '10051081',
 '10061114'}

## Female

In [15]:
SE_female = pickle.load(open('../Data/pandemic/SE_female.pk', 'rb'))
idd_female = [i not in drop_list for i in SE_female['SE']]
SE_female = SE_female[idd_female]
SE_female = SE_female.drop_duplicates('SE')
SE_female = SE_female[SE_female['2019_ROR'].notna()]

SE_female_2019 = SE_female[['SE','name','2019_A', '2019_B', '2020_A','2020_B','2019_ROR','2019_Delta']]
SE_female_2019['p_value'] = SE_female_2019.apply(lambda row: stats.fisher_exact([[row['2019_A'], row['2019_B']], [row['2020_A'], row['2020_B']]])[1], axis = 1)

# multipletests
SE_female_2019['sig'], SE_female_2019['p_corrected']  = multipletests(pvals=SE_female_2019['p_value'], 
                                                                  alpha=0.05, method='bonferroni')[0:2]
SE_female_2019_sig = SE_female_2019[SE_female_2019['sig']==True]
SE_female_2019_sig['CI_upper'] = SE_female_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[0], axis = 1)
SE_female_2019_sig['CI_lower'] = SE_female_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[1], axis = 1)


In [16]:
SE_female_2019_sig_over = SE_female_2019_sig[SE_female_2019_sig['2019_Delta']>0]
SE_female_2019_sig_under = SE_female_2019_sig[SE_female_2019_sig['2019_Delta']<0]

SE_female_2019_sig_under.sort_values('p_corrected', ascending=True).head()

,SE,name,2019_A,2019_B,2020_A,2020_B,2019_ROR,2019_Delta,p_value,sig,p_corrected,CI_upper,CI_lower
9704,10064848,chronic kidney disease,494,117471,194,113713,0.405691,-0.607287,5.921405e-29,True,4.772653e-25,0.479076,0.343547
8663,10061481,renal injury,185,117780,59,113848,0.329934,-0.681081,1.494678e-15,True,1.204711e-11,0.442328,0.246098
11115,10070608,infective pulmonary exacerbation of cystic fib...,378,117587,184,113723,0.503312,-0.513228,4.457624e-15,True,3.592845e-11,0.600392,0.421929
1029,10007617,cardio-respiratory arrest,249,117716,99,113808,0.411243,-0.602410,4.684562e-15,True,3.775757e-11,0.519156,0.325761
2312,10018687,granulocytopenia,61,117904,8,113899,0.135759,-0.868852,4.682178e-11,True,3.773835e-07,0.283704,0.064964


In [17]:
## Anything only occur in female but not in uncondition?
l_male = list(SE_male_2019_sig.SE)
l_female = list(SE_female_2019_sig.SE)
l_uncondition = list(SE_uncondition_2019_sig.SE)
set(l_female) - set(l_uncondition)

{'10003988',
 '10016825',
 '10022000',
 '10033553',
 '10034829',
 '10047899',
 '10048439',
 '10048621',
 '10054787',
 '10062237'}

# Conditioned on age


## Young (1-19 years)

In [18]:
SE_young = pickle.load(open('../Data/pandemic/SE_young.pk', 'rb'))
idd_young = [i not in drop_list for i in SE_young['SE']]
SE_young = SE_young[idd_young]

SE_young = SE_young.drop_duplicates('SE')
SE_young = SE_young[SE_young['2019_ROR'].notna()]
SE_young_2019 = SE_young[['SE','name','2019_A', '2019_B', '2020_A','2020_B','2019_ROR','2019_Delta']]
SE_young_2019['p_value'] = SE_young_2019.apply(lambda row: stats.fisher_exact([[row['2019_A'], row['2019_B']], [row['2020_A'], row['2020_B']]])[1], axis = 1)

# multipletests
SE_young_2019['sig'], SE_young_2019['p_corrected']  = multipletests(pvals=SE_young_2019['p_value'], alpha=0.05, method='bonferroni')[0:2]
SE_young_2019_sig = SE_young_2019[SE_young_2019['sig']==True]
SE_young_2019_sig['CI_upper'] = SE_young_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[0], axis = 1)
SE_young_2019_sig['CI_lower'] = SE_young_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[1], axis = 1)


In [19]:
SE_young_2019_sig_over = SE_young_2019_sig[SE_young_2019_sig['2019_Delta']>0]
SE_young_2019_sig_under = SE_young_2019_sig[SE_young_2019_sig['2019_Delta']<0]
SE_young_2019_sig_over.sort_values('p_corrected', ascending=True).head()

,SE,name,2019_A,2019_B,2020_A,2020_B,2019_ROR,2019_Delta,p_value,sig,p_corrected,CI_upper,CI_lower
1590,10020983,hypogammaglobulinaemia,4,7598,78,6528,22.696232,18.500000,1.412207e-21,True,4.493642e-18,62.026222,8.304857
2161,10029366,neutrophil count decreased,14,7588,90,6516,7.486188,5.428571,1.732967e-17,True,5.514301e-14,13.159375,4.258790
1904,10025256,lymphocyte count decreased,7,7595,69,6537,11.452501,8.857143,3.831113e-16,True,1.219060e-12,24.937961,5.259443
3849,10052015,cytokine release syndrome,20,7582,93,6513,5.413220,3.650000,4.513442e-15,True,1.436177e-11,8.785297,3.335453
4842,10061188,haematotoxicity,2,7600,49,6557,28.397133,23.500000,3.044908e-14,True,9.688899e-11,116.816549,6.903107


## Adult (20-65 years)

In [20]:
SE_adult = pickle.load(open('../Data/pandemic/SE_adult.pk', 'rb'))
idd_adult = [i not in drop_list for i in SE_adult['SE']]
SE_adult = SE_adult[idd_adult]

SE_adult = SE_adult.drop_duplicates('SE')

SE_adult = SE_adult[SE_adult['2019_ROR'].notna()]
SE_adult_2019 = SE_adult[['SE','name','2019_A', '2019_B', '2020_A','2020_B','2019_ROR','2019_Delta']]
SE_adult_2019['p_value'] = SE_adult_2019.apply(lambda row: stats.fisher_exact([[row['2019_A'], row['2019_B']], [row['2020_A'], row['2020_B']]])[1], axis = 1)

# multipletests
SE_adult_2019['sig'], SE_adult_2019['p_corrected']  = multipletests(pvals=SE_adult_2019['p_value'], alpha=0.05, method='bonferroni')[0:2]
SE_adult_2019_sig = SE_adult_2019[SE_adult_2019['sig']==True]
SE_adult_2019_sig['CI_upper'] = SE_adult_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[0], axis = 1)
SE_adult_2019_sig['CI_lower'] = SE_adult_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[1], axis = 1)


SE_adult_2019_sig_over = SE_adult_2019_sig[SE_adult_2019_sig['2019_Delta']>0]
SE_adult_2019_sig_under = SE_adult_2019_sig[SE_adult_2019_sig['2019_Delta']<0]
SE_adult_2019_sig_over.sort_values('p_corrected', ascending=True).head()

,SE,name,2019_A,2019_B,2020_A,2020_B,2019_ROR,2019_Delta,p_value,sig,p_corrected,CI_upper,CI_lower
12581,10084268,covid-19,0,74209,651,65607,inf,inf,6.592984e-214,True,4.807604e-210,inf,inf
4180,10038669,respiratory arrest,23,74186,412,65846,20.181895,16.913043,1.743776e-103,True,1.271562e-99,30.718280,13.259495
142,10001551,alanine aminotransferase increased,172,74037,683,65575,4.483351,2.970930,1.735457e-86,True,1.265495e-82,5.301086,3.791759
947,10007515,cardiac arrest,313,73896,751,65507,2.706630,1.399361,4.120472e-54,True,3.004648e-50,3.089439,2.371254
379,10003481,aspartate aminotransferase increased,158,74051,484,65774,3.448776,2.063291,3.134866e-48,True,2.285944e-44,4.128491,2.880969


## Elderly (<65 years)

In [21]:
SE_elderly = pickle.load(open('../Data/pandemic/SE_elderly.pk', 'rb'))
idd_elderly = [i not in drop_list for i in SE_elderly['SE']]
SE_elderly = SE_elderly[idd_elderly]

SE_elderly = SE_elderly.drop_duplicates('SE')
SE_elderly = SE_elderly[SE_elderly['2019_ROR'].notna()]
SE_elderly_2019 = SE_elderly[['SE','name','2019_A', '2019_B', '2020_A','2020_B','2019_ROR','2019_Delta']]
SE_elderly_2019['p_value'] = SE_elderly_2019.apply(lambda row: stats.fisher_exact([[row['2019_A'], row['2019_B']], [row['2020_A'], row['2020_B']]])[1], axis = 1)

# multipletests
SE_elderly_2019['sig'], SE_elderly_2019['p_corrected']  = multipletests(pvals=SE_elderly_2019['p_value'], alpha=0.05, method='bonferroni')[0:2]
SE_elderly_2019_sig = SE_elderly_2019[SE_elderly_2019['sig']==True]
SE_elderly_2019_sig['CI_upper'] = SE_elderly_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[0], axis = 1)
SE_elderly_2019_sig['CI_lower'] = SE_elderly_2019_sig.apply(lambda row: CI(row['2019_ROR'], row['2019_A'], row['2019_B'],row['2020_A'], row['2020_B'])[1], axis = 1)


SE_elderly_2019[SE_elderly_2019.name=='pyrexia']

,SE,name,2019_A,2019_B,2020_A,2020_B,2019_ROR,2019_Delta,p_value,sig,p_corrected
3435,10037660,pyrexia,617,45300,639,45732,1.025873,0.035656,0.670009,False,1.0


In [22]:
SE_elderly_2019_sig_over = SE_elderly_2019_sig[SE_elderly_2019_sig['2019_Delta']>0]
SE_elderly_2019_sig_under = SE_elderly_2019_sig[SE_elderly_2019_sig['2019_Delta']<0]


SE_elderly_2019_sig_under.sort_values('p_corrected', ascending=True).head()

,SE,name,2019_A,2019_B,2020_A,2020_B,2019_ROR,2019_Delta,p_value,sig,p_corrected,CI_upper,CI_lower
1471,10014866,enteritis,47,45870,12,46359,0.252626,-0.744681,0.000003,True,0.016784,0.476264,0.134001
2790,10028813,nausea,1850,44067,1601,44770,0.851816,-0.134595,0.000004,True,0.024666,0.911922,0.795672
3343,10036975,prostatic specific antigen increased,65,45852,23,46348,0.350059,-0.646154,0.000005,True,0.030224,0.563297,0.217544
1675,10017413,full blood count decreased,211,45706,129,46242,0.604288,-0.388626,0.000006,True,0.034343,0.752567,0.485224
2188,10022004,influenza like illness,177,45740,103,46268,0.575280,-0.418079,0.000006,True,0.037556,0.733701,0.451065


# Save all the populations in disproportionality estimation

In [23]:
condition_list = ['SE_uncondition_2019_sig_over', 'SE_uncondition_2019_sig_under', 'SE_male_2019_sig_over', 'SE_male_2019_sig_under',
                 'SE_female_2019_sig_over', 'SE_female_2019_sig_under', 
                 'SE_young_2019_sig_over', 'SE_young_2019_sig_under', 'SE_adult_2019_sig_over', 'SE_adult_2019_sig_under',
                 'SE_elderly_2019_sig_over', 'SE_elderly_2019_sig_under']

for condition in condition_list:    
    pickle.dump(locals()[condition], open('../Data/pandemic/results/'+condition+'_step1.pk', 'wb'))
    print(condition,'saved')
    

SE_uncondition_2019_sig_over saved
SE_uncondition_2019_sig_under saved
SE_male_2019_sig_over saved
SE_male_2019_sig_under saved
SE_female_2019_sig_over saved
SE_female_2019_sig_under saved
SE_young_2019_sig_over saved
SE_young_2019_sig_under saved
SE_adult_2019_sig_over saved
SE_adult_2019_sig_under saved
SE_elderly_2019_sig_over saved
SE_elderly_2019_sig_under saved
